# パスワードの強さを確認しよう

まずは、データセットを読み込んで、確認しましょう。

※：データセットのURL:
https://www.kaggle.com/bhavikbb/password-strength-classifier-dataset

In [1]:
# 生のデータセットを読み込む
def load_raw_dataset():

    import csv
    
    raw_dataset = []

    csv_file = open("data/password_strength.csv", newline='', encoding="utf-8_sig")
    reader = csv.reader(csv_file)

    # csv ファイルの各行を確認
    for line in reader:
         
        password = line[0]
        strength = line[1]
        
        # 強さが数値じゃなければ、使わない
        if not strength.isnumeric():
            continue
    
        raw_dataset.append((password, int(strength)))

    return raw_dataset

In [2]:
# 読み込んで、最初の20件を表示
raw_dataset = load_raw_dataset()
print(raw_dataset[0:20])

[('kzde5577', 1), ('kino3434', 1), ('visi7k1yr', 1), ('megzy123', 1), ('lamborghin1', 1), ('AVYq1lDE4MgAZfNt', 2), ('u6c8vhow', 1), ('v1118714', 1), ('universe2908', 1), ('as326159', 1), ('asv5o9yu', 1), ('612035180tok', 1), ('jytifok873', 1), ('WUt9IZzE0OQ7PkNE', 2), ('jerusalem393', 1), ('g067057895', 1), ('52558000aaa', 1), ('idofo673', 1), ('6975038lp', 1), ('sbl571017', 1)]


## パスワードの数値化：特徴を抽出

文字のままでAIを学習できないので、我々AIエンジニアとして文字データを数値化しないといけない。

このため、パスワードの「特徴」を考えて、数値化しましょう。単純にすると、５つの特徴が考えられる：

- パスワードの文字数：正規化された長さ（０～１）
- 小文字があるか？（0 か 1）
- 大文字があるか？（0 か 1）
- 数字キャラクターがあるか？（0 か 1）
- 記号があるか？（0 か 1）

In [3]:
# パスワードの長さ（正規化）を返す
def get_password_length(password):
    
    # パスワードの最大の長さ
    max_password_length = 20

    # 長さを求める
    password_length = len(password)
    
    # 最大を超えないように
    password_length = min(password_length, max_password_length)
    
    # 正規化（0 ～ 1）
    password_length = password_length / max_password_length
    
    return password_length

In [4]:
print(get_password_length("abcdefghij"))

0.5


In [5]:
# 正規表現を使う
import re

# 小文字あるか？
def has_small(text):
    small = "[a-z]"

    if re.search(small, text) is None:
        return 0
    else:
        return 1

In [6]:
print(has_small("HELLO"))
print(has_small("HeLLO"))

0
1


In [7]:
# 大文字あるか？
def has_large(text):
    large = "[A-Z]"

    if re.search(large, text) is None:
        return 0
    else:
        return 1

In [8]:
print(has_large("hello"))
print(has_large("hEllo"))

0
1


In [9]:
# 数値がある場合は「1」ない場合は「0」を返す
def has_numbers(text):
    
    numbers = "[1234567890]"
    
    if re.search(numbers, text) is None:
        return 0
    else:
        return 1

In [10]:
print(has_numbers("hello"))
print(has_numbers("h3llo"))

0
1


In [11]:
# 記号がある場合は「1」ない場合は「0」を返す
def has_symbols(text):
    
    symbols = "[-!$%^&*()_+|~=`{}\[\]:\";'<>?,.\/@]"

    if re.search(symbols, text) is None:
        return 0
    else:
        return 1

In [12]:
print(has_symbols("hello"))
print(has_symbols("h?llo"))

0
1


## パスワードを特徴ベクトルに変換

上記の３つの関数を使って、文字列をAIに入力できるベクトルに変換

In [13]:
def password_to_features(password):
    
    x = [0, 0, 0, 0, 0]

    x[0] = get_password_length(password)
    x[1] = has_small(password)
    x[2] = has_large(password)
    x[3] = has_numbers(password)
    x[4] = has_symbols(password)
    
    return x

In [14]:
print(password_to_features("hello"))
print(password_to_features("h3llo"))
print(password_to_features("h3?lo!!"))

[0.25, 1, 0, 0, 0]
[0.25, 1, 0, 1, 0]
[0.35, 1, 0, 1, 1]


**出力も、「one-hot encoding」でベクトル化しましょう。**

In [15]:
def strength_to_onehot(strength):
    
    y = [0.0, 0.0, 0.0]
    y[strength] = 1.0
    
    return y

In [16]:
print(strength_to_onehot(0))
print(strength_to_onehot(1))
print(strength_to_onehot(2))

[1.0, 0.0, 0.0]
[0.0, 1.0, 0.0]
[0.0, 0.0, 1.0]


## データセットの前処理

これでrawのデータをAIに入力できる形にできる

In [17]:
import numpy as np

def prepare_dataset(raw_dataset):
    
    data_x = []
    data_y = []
    
    for line in raw_dataset:
        
        password = line[0]
        strength = line[1]

        x = password_to_features(password)
        y = strength_to_onehot(strength)
    
        data_x.append(x)
        data_y.append(y)

    data_x = np.array(data_x)
    data_y = np.array(data_y)
    
    return data_x, data_y

data_x, data_y = prepare_dataset(raw_dataset)

In [18]:
print(data_x[0:20])
print(data_y[0:20])

[[0.4  1.   0.   1.   0.  ]
 [0.4  1.   0.   1.   0.  ]
 [0.45 1.   0.   1.   0.  ]
 [0.4  1.   0.   1.   0.  ]
 [0.55 1.   0.   1.   0.  ]
 [0.8  1.   1.   1.   0.  ]
 [0.4  1.   0.   1.   0.  ]
 [0.4  1.   0.   1.   0.  ]
 [0.6  1.   0.   1.   0.  ]
 [0.4  1.   0.   1.   0.  ]
 [0.4  1.   0.   1.   0.  ]
 [0.6  1.   0.   1.   0.  ]
 [0.5  1.   0.   1.   0.  ]
 [0.8  1.   1.   1.   0.  ]
 [0.6  1.   0.   1.   0.  ]
 [0.5  1.   0.   1.   0.  ]
 [0.55 1.   0.   1.   0.  ]
 [0.4  1.   0.   1.   0.  ]
 [0.45 1.   0.   1.   0.  ]
 [0.45 1.   0.   1.   0.  ]]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


## モデルを作成し、学習しましょう！

In [19]:
from keras.layers import Input, Dense
from keras.models import Model

def create_model():
    
    lyr_input = Input(shape=(5,))
    lyr_hiddn = Dense(units=10, activation="relu")(lyr_input)
    lyr_hiddn = Dense(units=10, activation="relu")(lyr_hiddn)
    lyr_outpt = Dense(units=3, activation="softmax")(lyr_hiddn)
    
    model = Model(inputs=lyr_input, outputs=lyr_outpt)
    model.summary()
    
    model.compile(optimizer="adam", loss="categorical_crossentropy")
    
    return model
    
model = create_model()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense (Dense)                (None, 10)                60        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 33        
Total params: 203
Trainable params: 203
Non-trainable params: 0
_________________________________________________________________


In [20]:
# 学習開始
# 検証用のデータ割合：20%
model.fit(data_x, data_y, epochs=5, batch_size=50, validation_split=0.2)

Epoch 1/5
10715/10715 [==============================] - 6s 517us/step - loss: 0.0644 - val_loss: 0.0014
Epoch 2/5
10715/10715 [==============================] - 5s 508us/step - loss: 8.4842e-04 - val_loss: 0.0011
Epoch 3/5
10715/10715 [==============================] - 5s 511us/step - loss: 5.2661e-04 - val_loss: 5.3694e-04
Epoch 4/5
10715/10715 [==============================] - 6s 523us/step - loss: 3.7913e-04 - val_loss: 3.3701e-04
Epoch 5/5
10715/10715 [==============================] - 5s 499us/step - loss: 2.8710e-04 - val_loss: 3.4630e-04


## では、パスワードを確認しましょう

In [31]:
np.set_printoptions(suppress = True)

def test_password(password):
    
    # １つのパスワードを確認
    x = password_to_features(password)
    x = [x]
    x = np.array(x)
    
    # 推定
    y = model.predict(x)[0]

    print("Prediction:", y)
    
    # もっとも信頼性が高いものがどれ？
    best = np.argmax(y)
    confidence = y[best] * 100
    
    if (best == 0):
        print("Weak password!")
    elif (best == 1):
        print("Normal password")
    else:
        print("Strong password!!")
        
    # 信頼性も表示する
    print("Confidence:", confidence)

In [32]:
test_password("hello")

Prediction: [0.99999905 0.00000098 0.        ]
Weak password!
Confidence: 99.99990463256836


In [33]:
test_password("1sdssig3!")

Prediction: [0.         0.9999651  0.00003493]
Normal password
Confidence: 99.99650716781616


In [34]:
test_password("8m9a79ds&)da(@asd!aas3$")

Prediction: [0. 0. 1.]
Strong password!!
Confidence: 100.0


In [36]:
test_password("aaaaaaaaaaaaaaaaaaaaaaA")

Prediction: [0.         0.000001   0.99999905]
Strong password!!
Confidence: 99.99990463256836


## 議論、宿題：どう改善できると思う？